In [35]:
import pandas as pd
import numpy as np
from os.path import join
import functions
from collections import defaultdict, deque

In [4]:
##############################
company = "솔림"
cutoff = "250831"
##############################
path_read_parquet = join(r"D:\3.자산\전산 dataset\솔림",cutoff,"parquet")

In [76]:
# 전체원데이터와 새보증인상태 파일 읽기
    # 읽을 칼럼
cols_to_read_account = ["채무자키","계좌키","성명","주민등록번호","원채권사","매각사구분","채권상태","상태구분2","담당자","배정가능여부_통합"]
cols_to_read_grt = ["채무자키","계좌키","보증인성명인","주민번호인","원채권사","매각사구분","새보증인상태","담당자",]

cols_to_change = {"배정가능여부_통합":"배정가능여부", "보증인성명인":"성명","주민번호인":"주민등록번호","상태구분2":"채무자상태","새보증인상태":"채무자상태"}

account = pd.read_parquet(functions.키워드로파일명찾기(폴더=path_read_parquet, 포함키워드="전체_원데이터", 전체경로=True, 여러파일허용=False), columns=cols_to_read_account)
grt = pd.read_parquet(functions.키워드로파일명찾기(폴더=path_read_parquet, 포함키워드="새보증인상태", 전체경로=True, 여러파일허용=False), columns=cols_to_read_grt)

# 칼럼정리
account = account.rename(columns=cols_to_change)
grt = grt.rename(columns=cols_to_change)

account["채무구분"] = "차주"
account["배정가능여부"] = account["배정가능여부"].replace(["nan", np.nan], "불가")
grt["채무구분"] = "보증인"


# 통합하기
df = pd.concat([account, grt])

# ""는 na로
df = df.replace("",np.nan)

# 주민번호 없는 건 삭제하기
df = df.dropna(subset=["주민등록번호"])

# 비어있는 채권상태 값 넣기
    # 빈 문자열을 NaN으로 바꾸기 (있으면)
df["채권상태"] = df["채권상태"].replace("", np.nan)

    # 같은 계좌키 그룹에서 첫 번째 채권상태를 가져와서 채우기
df["채권상태"] = df["채권상태"].fillna(
    df.groupby("계좌키")["채권상태"].transform("first")
)

df["배정가능여부"] = df["배정가능여부"].fillna(
    df.groupby("계좌키")["배정가능여부"].transform("first")
)

In [77]:
# 고유채무자키수 작성하기
주민등록번호별_고유채무자키수 = df.groupby('주민등록번호')["채무자키"].nunique().reset_index().rename(columns={"채무자키":"고유채무자키수"})
df = df.merge(주민등록번호별_고유채무자키수, how='left', on='주민등록번호')

In [78]:
def make_groups(df):
    # 그래프 만들기
    graph = defaultdict(set)
    for _, row in df.iterrows():
        jumin = row["주민등록번호"]
        debtor = row["채무자키"]
        graph[jumin].add(debtor)
        graph[debtor].add(jumin)

    group_map = {}  # 노드별 그룹키
    group_id = 1

    # 모든 주민등록번호 탐색
    for jumin in df["주민등록번호"].unique():
        if jumin in group_map:  # 이미 그룹 배정됨
            continue

        # BFS 탐색
        queue = deque([jumin])
        while queue:
            node = queue.popleft()
            if node in group_map:
                continue
            group_map[node] = group_id
            for neighbor in graph[node]:
                if neighbor not in group_map:
                    queue.append(neighbor)

        group_id += 1

    # df에 그룹키 추가
    df["그룹키"] = df["주민등록번호"].map(group_map)
    return df

# 사용 예시
result = make_groups(df)

In [79]:
# 그룹키 format변경
df["그룹키"] = df["그룹키"].apply(lambda x: f"g{int(x):06d}")

In [ ]:
# 필요칼럼만, 중복제거
cols = ["그룹키","채무자키","채권상태","담당자","배정가능여부","원채권사","매각사구분"]
df_rst = df[cols].copy()
df_rst = df_rst.sort_values(by=cols)
df_rst = df_rst.drop_duplicates(subset=["그룹키","채무자키"], keep='first').reset_index(drop=True)



In [84]:
# 배정가능건들 중 다중담당자
배정가능건 = df_rst[df_rst["배정가능여부"]=="가능"]
다중담당자 = 배정가능건.groupby('그룹키').filter(lambda x : x.담당자.nunique()>1)

functions.save_df_to_excel_underline(다중담당자, "그룹_다중담당자.xlsx")

In [81]:
# 출력 : 오래걸리니까 key_columns_no 작성하지마
functions.save_df_to_excel_underline(df_rst, "담당자배정_그룹키.xlsx", key_columns_no=0)